In [2]:
import requests

## Question 1

First, we define our request URL with its query parameters. Using the Requests library, we send a GET request with the given URL and query parameters. I chose option 2 (to filter by type as we make the HTTP request) because it reduces the amount of data we have to process on our end. If we were to not use the filters we would just be re-writing functionality that the mbta API already offers when we're filtering through the response. The answer to question 1 is printed below the following code block:

In [5]:
URL = "https://api-v3.mbta.com/routes"
types = "0,1"
query_params = {'filter[type]': types}

request = requests.get(url = URL, params = query_params)

response = request.json()
for i in response['data']:
    print(i['attributes']['long_name'])

Red Line
Mattapan Trolley
Orange Line
Green Line B
Green Line C
Green Line D
Green Line E
Blue Line


## Question 2

As I read this question, I began to think it may be better to represent routes and stops as objects instead of having to iterate through the response from question 1. In order to find how many stops are on a given route, we need to make a GET request to endpoint `https://api-v3.mbta.com/stops?filter[route]={route_id}`. First, we define the class `Route` with attributes `id` and `long_name`

In [23]:
class Route:
    def __init__(self, _id, long_name):
        self._id = _id
        self.long_name = long_name
    def __str__(self):
        return f"[id: {self._id}, name: {self.long_name}]"

We then define an array `routes` which will hold all our `Route` objects.

In [25]:
routes = []
for i in response['data']:
    route = Route(i['id'], i['attributes']['long_name'])
    routes.append(route)

We then define a `Stop` class with attributes `id`, `name`, and `route`. Here, we see a many to one relationship. Multiple stops will belong to a single route (reflected by the `route` attribute on the `Stop` object). Vice-versa, a single route will have many stops.

In [26]:
class Stop:
    def __init__(self, _id, name, route):
        self._id = _id
        self.name = name
        self.route = route
        
    def __str__(self):
        return f"[id: {self._id}, name: {self.name}, route_id: {self.route}]"

We now have a list of `Route` objects which we can iterate through and send an HTTP GET Request for each route to find its given stops. As we get the stops for a given `Route`, we can keep track of how many stops there are per route using a dictionary `routes_with_stops` and every time we create a `Stop` object we can increment our count of stops for that `Route`. For part 3, we are interested in which stops have more than 1 route going through them. We can keep track of this in the same `for` loop using a dictionary called `stops_with_routes`. For each stop, we can append the route which it belongs to. Stops with multiple routes will be returned by each request for that route.

In [27]:
URL = "https://api-v3.mbta.com/stops"

stops = []
# For parts 1 and 2, dict with key: str (route), value: int (# of stops for that route)
num_stops_per_route = {}

# For part 3, dict with key: str (stop), 
# value: [int, list(str)]([# routes that stop here, [names of routes]])
routes_per_stop = {}

for route in routes:
    route_id = route._id

    query_params = {'filter[route]': route_id}
    request = requests.get(url = URL, params = query_params)
    response = request.json()
    
    num_stops_per_route[route_id] = 0
    
    for i in response['data']:
        stop = Stop(i['id'], i['attributes']['name'], route_id)
        stops.append(stop)
        # for each stop we increment our counter of stops for that route
        num_stops_per_route[route_id] += 1
        
        # check if we've seen this stop before
        if stop.name in routes_per_stop.keys():
            # if we have, increment its counter 
            # (meaning we have seen this stop for a different route)
            routes_per_stop[stop.name][0] += 1
            if route_id not in routes_per_stop[stop.name][1]:
                # add route to list of routes that pass through that stop
                routes_per_stop[stop.name][1].append(route_id) 
        else:
            # else we need to add this stop along with a count of 1 and 
            # the route that passes through it
            routes_per_stop[stop.name] = [1, [route_id]]


most_stops = float("-inf")
least_stops = float("inf")
route_most_stops = ""
route_least_stops = ""

# iterate through dict (key: route, val: # stops for that route) keeping track of which
# route has the most and least stops
for route,stop in num_stops_per_route.items():
    if stop > most_stops:
        most_stops = stop
        route_most_stops = route
    if stop < least_stops:
        least_stops = stop
        route_least_stops = route

### Part 1

In [29]:
print("Subway route with most stops: %s (%s stops)"%(route_most_stops, most_stops))

Subway route with most stops: Green-D (25 stops)


### Part 2

In [30]:
print("Subway route with least stops: %s (%s stops)"%(route_least_stops, least_stops))

Subway route with least stops: Mattapan (8 stops)


### Part 3

In [31]:
print("The following stops connect 2 or more subway routes:")
# iterate through dict (key: stop, value: [# routes that pass thru that stop, [route names that pass thru]])
# we are interested in the stops that have more than 1 route passing thru it
for stop, routes in routes_per_stop.items():
    if routes[0] > 1:
        print("stop: " + str(stop) + "; routes that connect: " + str(routes[1]))

The following stops connect 2 or more subway routes:
stop: Park Street; routes that connect: ['Red', 'Green-B', 'Green-C', 'Green-D', 'Green-E']
stop: Downtown Crossing; routes that connect: ['Red', 'Orange']
stop: Ashmont; routes that connect: ['Red', 'Mattapan']
stop: State; routes that connect: ['Orange', 'Blue']
stop: Haymarket; routes that connect: ['Orange', 'Green-D', 'Green-E']
stop: North Station; routes that connect: ['Orange', 'Green-D', 'Green-E']
stop: Government Center; routes that connect: ['Green-B', 'Green-C', 'Green-D', 'Green-E', 'Blue']
stop: Boylston; routes that connect: ['Green-B', 'Green-C', 'Green-D', 'Green-E']
stop: Arlington; routes that connect: ['Green-B', 'Green-C', 'Green-D', 'Green-E']
stop: Copley; routes that connect: ['Green-B', 'Green-C', 'Green-D', 'Green-E']
stop: Hynes Convention Center; routes that connect: ['Green-B', 'Green-C', 'Green-D']
stop: Kenmore; routes that connect: ['Green-B', 'Green-C', 'Green-D']
stop: Science Park/West End; routes 

## Question 3

For this question, we want to find out how many times one must transfer lines in order to get from stop A to stop B. First, I find the `Stop` object based on the inputed string. Then I get the route that stop belongs to. Conveniently, in question 2 we found which stops have more than 1 route passing through it meaning these stops are our points of transfer. If our start route is the same as our end route then we do not need to switch lines. Otherwise, we need to check which stop we can get to on the same route that allows us to transfer to another line. We do this by keeping track of which route we are currently on and which routes we have already seen. If I am on the green line, I need to find all the stops that have more than 1 line passing through which will allow me to transfer. We check each stop by using our `routes_per_stop` dictionary which gives us other routes we can take from that stop. We want to try the routes we haven't already tried other than the route we came from. Each time we try a route, this simulates transfering to another line so we update our starting location every time we transfer. We know we have reached our destination when our starting location is the same as our ending location.

In [32]:
def getStop(stop):
    for i in stops:
        if stop in i.name:
            return i

def findRoute(start, end, res):
    # get the route we start on and want to end on
    start_route = getStop(start).route
    end_route = getStop(end).route
    
    # call our helper method to calculate how many times 
    # we transfer before reaching our destination
    res = findAllRoutes(start_route, end_route, res, set())
    
    # format the result according to Google doc examples
    green_line = False
    ans = ""
    for i in res:
        if "Green" in i:
            if not green_line:
                ans += "Greenline, "
                green_line = True
        else:
            ans += i + "line, "
    return ans[:-2]
        
    
def findAllRoutes(start_route, end_route, res, seen):
    # if we are on the same route as our destination stop then we return
    if start_route == end_route:
        res.append(end_route)
        return res
    # else we need to check if we can get to our destination 
    # by transfering to what's available to us
    for stop, routes in routes_per_stop.items():
        # we can only transfer at stops with more than 1 route passing through
        connections = routes[1]
        if len(connections) > 1:
            # if our starting route (line) is in the current 
            # stop's routes then we can transfer here
            if start_route in connections:
                # we check each route we can transfer to
                for route in connections:
                    # we skip the route we are already on and 
                    # also skip routes we've already checked
                    if route != start_route and route not in seen:
                        res.append(start_route)
                        seen.add(start_route)
                        # our new start is the route to which we transfer to
                        start_route = route
                        return findAllRoutes(start_route, end_route, res, seen)    

There are 3 testcases we should test:
    1. Start and stop are on the same route
    2. Start and stop require 1 transfer
    3. Start and stop require more than 1 transfer

Testcase #1

In [33]:
print(findRoute("Davis", "Kendall", []))

Redline


Testcase #2

In [34]:
print(findRoute("Ashmont", "Arlington", []))

Redline, Greenline


Testcase #3

In [15]:
print(findRoute("Airport", "Central", []))

Blueline, Orangeline, Redline


I used the following map to come up with test cases and be able to visualize how to get from one stop to another: https://commonwealthmagazine.org/wp-content/uploads/2019/01/mbta-map.png
In the following line of code, replace stops "Airport" and "Central" to test whichever two stops. Then hit `Control` + `Enter` to run the line of code and see the results.

In [ ]:
print(findRoute("Airport", "Central", []))